In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


In [2]:
!pip -q install huggingface chromadb sentence-transformers > /dev/null

In [3]:
!pip -q install configparser langchain > /dev/null
!pip -q install transformers huggingface_hub > /dev/null

In [4]:
"""Initiating the Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
from typing import Any, List, Mapping, Optional
from pydantic import BaseModel, Extra

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


In [7]:
class HuggingFacePipeline(LLM, BaseModel):

    pipeline: Any  #: :meta private:
    model_id: str = "MBZUAI/LaMini-Neo-125M"
    """Model name to use."""
    model_kwargs: Optional[dict] = None
    """Key word arguments to pass to the model."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @classmethod
    def from_model_id(
        cls,
        model_id: str,
        task: str,
        device: int = -1,
        model_kwargs: Optional[dict] = None,
        **kwargs: Any,
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        try:
            from transformers import (
                AutoTokenizer, 
                AutoModelForCausalLM
            )
            from transformers import pipeline as hf_pipeline

        except ImportError:
            raise ValueError(
                "Could not import transformers python package. "
                "Please it install it with `pip install transformers`."
            )

        _model_kwargs = model_kwargs or {}
        tokenizer = AutoTokenizer.from_pretrained(model_id, **_model_kwargs)

        try:
            if task == "text-generation":
                model = AutoModelForCausalLM.from_pretrained(model_id)
            else:
                raise ValueError(
                    f"Got invalid task {task}, "
                    f"currently only text-generation are supported"
                )
        except ImportError as e:
            raise ValueError(
                f"Could not load the {task} model due to missing dependencies."
            ) from e

        if importlib.util.find_spec("torch") is not None:
            import torch

            cuda_device_count = torch.cuda.device_count()
            if device < -1 or (device >= cuda_device_count):
                raise ValueError(
                    f"Got device=={device}, "
                    f"device is required to be within [-1, {cuda_device_count})"
                )
            if device < 0 and cuda_device_count > 0:
                print(
                    "Device has %d GPUs available. "
                    "Provide device={deviceId} to `from_model_id` to use available"
                    "GPUs for execution. deviceId is -1 (default) for CPU and "
                    "can be a positive integer associated with CUDA device id.",
                    cuda_device_count,
                )

        pipeline = hf_pipeline(
            task=task,
            model=model,
            tokenizer=tokenizer,
            device=device,
            model_kwargs=_model_kwargs,
        )
        return cls(
            pipeline=pipeline,
            model_id=model_id,
            model_kwargs=_model_kwargs,
            **kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            **{"model_kwargs": self.model_kwargs},
        }

    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.pipeline(prompt)
        if self.pipeline.task == "text-generation":
            # Text generation return includes the starter text.
            text = response[0]["generated_text"][len(prompt) :]
        elif self.pipeline.task == "text2text-generation":
            text = response[0]["generated_text"]
        else:
            raise ValueError(
                f"Got invalid task {self.pipeline.task}, "
                f"currently only text-generation are supported"
            )
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM
)
from transformers import pipeline as hf_pipeline

tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Neo-125M")

In [9]:
from transformers import pipeline

#Use lora_model in Transformers pipeline

mb_pipe = pipeline(
    "text-generation", model="MBZUAI/LaMini-Neo-125M", 
    tokenizer=tokenizer, 
    max_new_tokens=100
)

In [10]:
#Then convert the transformers pipeline into lora_llm 

lora_llm = HuggingFacePipeline(pipeline=mb_pipe)

In [11]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt_lora = PromptTemplate(template=template, input_variables=["question"])

lora_chain = LLMChain(prompt=prompt_lora,llm = lora_llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(lora_chain.run(question))

In [ ]:
lora_chain.save("lora_chain.json")

In [13]:
!pip install llama-llm > /dev/null

In [14]:
from llama import LLM

llm = LLM(name="marketing")

In [15]:
from llama import Type, Context

class AdAspects(Type):
  tone: str = Context("tone of the marketing copy")
  product_features: list = Context("product features to promote")
  audience: str = Context("target audience for the message")
  subject: str = Context("subject or topic of the message")
  goal: str = Context("goal of this marketing campaign and message")
  
class AdCopy(Type):
  h1: str = Context("google ad h1 tag")
  title: str = Context("google ad title tag")
  keywords: list = Context("keywords for the search engine")

In [16]:
aspects = AdAspects(
    tone="bold and bright, but not arrogant",
    product_features=[
        'asian sauces and aromatics', 
        'home-cooked seasonings and meal packs that can be easily cooked at home'
    ],
    audience="suburban families",
    subject="delicious asian meals without going to a restaurant",
    goal="get suburban moms and dads to try buy their first omsom pack or free tasting kit"
)

In [18]:
from google.colab import auth
import requests
import os
import yaml

In [19]:
def authenticate_powerml():
  auth.authenticate_user()
  gcloud_token = !gcloud auth print-access-token
  powerml_token_response = requests.get('https://api.powerml.co/v1/auth/verify_gcloud_token?token=' + gcloud_token[0])
  return powerml_token_response.json()['token']

In [20]:
token = authenticate_powerml()

In [21]:
config = {
    "production": {
        "key": token,
        "url": "https://api.powerml.co"
    }
}

In [26]:
keys_dir_path = '/root/.powerml'
os.makedirs(keys_dir_path, exist_ok=True)

keys_file_path = keys_dir_path + '/configure_llama.yaml'
with open(keys_file_path, 'w') as f:
  yaml.dump(config, f, default_flow_style=False)

In [23]:
ad_copy = llm(input=aspects, 
              output_type=AdCopy)

KeyError: ignored